In [1]:
dataset_name_list_128 = [
'ACSF1',
'Adiac',
'AllGestureWiimoteX',
'AllGestureWiimoteY',
'AllGestureWiimoteZ',
'ArrowHead',
'Beef',
'BeetleFly',
'BirdChicken',
'BME',
'Car',
'CBF',
'Chinatown',
'ChlorineConcentration',
'CinCECGTorso',
'Coffee',
'Computers',
'CricketX',
'CricketY',
'CricketZ',
'Crop',
'DiatomSizeReduction',
'DistalPhalanxOutlineAgeGroup',
'DistalPhalanxOutlineCorrect',
'DistalPhalanxTW',
'DodgerLoopDay',
'DodgerLoopGame',
'DodgerLoopWeekend',
'Earthquakes',
'ECG200',
'ECG5000',
'ECGFiveDays',
'ElectricDevices',
'EOGHorizontalSignal',
'EOGVerticalSignal',
'EthanolLevel',
'FaceAll',
'FaceFour',
'FacesUCR',
'FiftyWords',
'Fish',
'FordA',
'FordB',
'FreezerRegularTrain',
'FreezerSmallTrain',
'Fungi',
'GestureMidAirD1',
'GestureMidAirD2',
'GestureMidAirD3',
'GesturePebbleZ1',
'GesturePebbleZ2',
'GunPoint',
'GunPointAgeSpan',
'GunPointMaleVersusFemale',
'GunPointOldVersusYoung',
'Ham',
'HandOutlines',
'Haptics',
'Herring',
'HouseTwenty',
'InlineSkate',
'InsectEPGRegularTrain',
'InsectEPGSmallTrain',
'InsectWingbeatSound',
'ItalyPowerDemand',
'LargeKitchenAppliances',
'Lightning2',
'Lightning7',
'Mallat',
'Meat',
'MedicalImages',
'MelbournePedestrian',
'MiddlePhalanxOutlineAgeGroup',
'MiddlePhalanxOutlineCorrect',
'MiddlePhalanxTW',
'MixedShapesRegularTrain',
'MixedShapesSmallTrain',
'MoteStrain',
'NonInvasiveFetalECGThorax1',
'NonInvasiveFetalECGThorax2',
'OliveOil',
'OSULeaf',
'PhalangesOutlinesCorrect',
'Phoneme',
'PickupGestureWiimoteZ',
'PigAirwayPressure',
'PigArtPressure',
'PigCVP',
'PLAID',
'Plane',
'PowerCons',
'ProximalPhalanxOutlineAgeGroup',
'ProximalPhalanxOutlineCorrect',
'ProximalPhalanxTW',
'RefrigerationDevices',
'Rock',
'ScreenType',
'SemgHandGenderCh2',
'SemgHandMovementCh2',
'SemgHandSubjectCh2',
'ShakeGestureWiimoteZ',
'ShapeletSim',
'ShapesAll',
'SmallKitchenAppliances',
'SmoothSubspace',
'SonyAIBORobotSurface1',
'SonyAIBORobotSurface2',
'StarLightCurves',
'Strawberry',
'SwedishLeaf',
'Symbols',
'SyntheticControl',
'ToeSegmentation1',
'ToeSegmentation2',
'Trace',
'TwoLeadECG',
'TwoPatterns',
'UMD',
'UWaveGestureLibraryAll',
'UWaveGestureLibraryX',
'UWaveGestureLibraryY',
'UWaveGestureLibraryZ',
'Wafer',
'Wine',
'WordSynonyms',
'Worms',
'WormsTwoClass',
'Yoga'
]

In [2]:
import _ucrdtw
import numpy as np
import matplotlib.pyplot as plt
import time
from os.path import dirname
from sklearn import preprocessing
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score
import os


def set_nan_to_zero(a):
    where_are_NaNs = np.isnan(a)
    a[where_are_NaNs] = 0
    return a

def TSC_data_loader(dataset_path,dataset_name):
    Train_dataset = np.loadtxt(
        dataset_path + '/' + dataset_name + '/' + dataset_name + '_TRAIN.tsv')
    Test_dataset = np.loadtxt(
        dataset_path + '/' + dataset_name + '/' + dataset_name + '_TEST.tsv')
    Train_dataset = Train_dataset.astype(np.float32)
    Test_dataset = Test_dataset.astype(np.float32)

    X_train = Train_dataset[:, 1:]
    y_train = Train_dataset[:, 0:1]

    X_test = Test_dataset[:, 1:]
    y_test = Test_dataset[:, 0:1]
    le = preprocessing.LabelEncoder()
    le.fit(np.squeeze(y_train, axis=1))
    y_train = le.transform(np.squeeze(y_train, axis=1))
    y_test = le.transform(np.squeeze(y_test, axis=1))
    return set_nan_to_zero(X_train), y_train, set_nan_to_zero(X_test), y_test

def get_big_matrix(query_list, data_list):
    Result_matrix = np.zeros((query_list.shape[0],data_list.shape[0]))
    for i in range(query_list.shape[0]):
        query = query_list[i]
        for j in range(data_list.shape[0]):
            data = data_list[j]
            loc, dist = _ucrdtw.ucrdtw(data, query, 0.05, True)
            Result_matrix[i,j] = dist
    return Result_matrix

def predict_with_matrix(matrix,label):
    min_index = np.argmin(matrix, axis=-1)
    
    result = label[min_index]
    return result


dataset_path = dirname("/home1/tangw/Desktop/Datasets/UCRArchive_2018/")
distance_matrix_log_folder =dirname("./distance_matrix_of_DTW/")

if not(os.path.exists(distance_matrix_log_folder)):
    os.mkdir(distance_matrix_log_folder)


for dataset_name in dataset_name_list_128:
    result_dict = {}
    X_train, y_train, X_test, y_test = TSC_data_loader(dataset_path, dataset_name)
    
    train_test_result_matrix = get_big_matrix(X_test, X_train)
    
    train_train_result_matrix = get_big_matrix(X_train, X_train)

    result_dict['train_train'] = train_train_result_matrix
    result_dict['test_train'] = train_test_result_matrix
    
    save_log_path = os.path.join(distance_matrix_log_folder, dataset_name)
    
    if not(os.path.exists(save_log_path)):
        os.mkdir(save_log_path)
    save_log_path = save_log_path +'/'+ dataset_name+'.npy'
    np.save(save_log_path, result_dict)
    print(dataset_name,'finish')

ACSF1 finish
Adiac finish
AllGestureWiimoteX finish
AllGestureWiimoteY finish
AllGestureWiimoteZ finish
ArrowHead finish
Beef finish
BeetleFly finish
BirdChicken finish
BME finish
Car finish
CBF finish
Chinatown finish
ChlorineConcentration finish
CinCECGTorso finish
Coffee finish
Computers finish
CricketX finish
CricketY finish
CricketZ finish
Crop finish
DiatomSizeReduction finish
DistalPhalanxOutlineAgeGroup finish
DistalPhalanxOutlineCorrect finish
DistalPhalanxTW finish
DodgerLoopDay finish
DodgerLoopGame finish
DodgerLoopWeekend finish
Earthquakes finish
ECG200 finish
ECG5000 finish
ECGFiveDays finish
ElectricDevices finish
EOGHorizontalSignal finish
EOGVerticalSignal finish
EthanolLevel finish
FaceAll finish
FaceFour finish
FacesUCR finish
FiftyWords finish
Fish finish
FordA finish
FordB finish
FreezerRegularTrain finish
FreezerSmallTrain finish
Fungi finish
GestureMidAirD1 finish
GestureMidAirD2 finish
GestureMidAirD3 finish
GesturePebbleZ1 finish
GesturePebbleZ2 finish
GunPoin

In [7]:
dataset_name = 'ACSF1'
result = np.load(distance_matrix_log_folder+'/'+dataset_name+'/'+dataset_name+'.npy')
print(result.item().get('train_train').shape)
print(result.item().get('test_train').shape)

(100, 100)
(100, 100)


In [8]:
X_train, y_train, X_test, y_test = TSC_data_loader(dataset_path, dataset_name)
test_train_matrix = result.item().get('test_train')
y_predict = y_train[np.argmin(test_train_matrix,-1)]
print(y_predict.shape)
acc = accuracy_score(y_predict,y_test)
print(acc)

(100,)
0.64


In [10]:
for dataset_name in dataset_name_list_128:
    X_train, y_train, X_test, y_test = TSC_data_loader(dataset_path, dataset_name)
    result = np.load(distance_matrix_log_folder+'/'+dataset_name+'/'+dataset_name+'.npy')
    test_train_matrix = result.item().get('test_train')
    y_predict = y_train[np.argmin(test_train_matrix,-1)]

    acc = accuracy_score(y_predict,y_test)
    print(dataset_name,acc)

ACSF1 0.64
Adiac 0.6061381074168798
AllGestureWiimoteX 0.7271428571428571
AllGestureWiimoteY 0.7442857142857143
AllGestureWiimoteZ 0.6428571428571429
ArrowHead 0.7314285714285714
Beef 0.6666666666666666
BeetleFly 0.7
BirdChicken 0.7
BME 0.98
Car 0.7333333333333333
CBF 0.98
Chinatown 0.9565217391304348
ChlorineConcentration 0.6484375
CinCECGTorso 0.8260869565217391
Coffee 1.0
Computers 0.52
CricketX 0.7564102564102564
CricketY 0.7512820512820513
CricketZ 0.7461538461538462
Crop 0.6904166666666667
DiatomSizeReduction 0.9640522875816994
DistalPhalanxOutlineAgeGroup 0.7697841726618705
DistalPhalanxOutlineCorrect 0.7101449275362319
DistalPhalanxTW 0.6115107913669064
DodgerLoopDay 0.5125
DodgerLoopGame 0.8768115942028986
DodgerLoopWeekend 0.9710144927536232
Earthquakes 0.6906474820143885
ECG200 0.9
ECG5000 0.9242222222222222
ECGFiveDays 0.8164924506387921
ElectricDevices 0.6248216833095578
EOGHorizontalSignal 0.4972375690607735
EOGVerticalSignal 0.5248618784530387
EthanolLevel 0.274
FaceAll 